In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
from tqdm import tqdm
from datetime import timedelta
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

## 专门用来处理网格强度数据

In [2]:
### 北京市重点区域信息
cols = ['id', 'area_name', 'area_type', 'Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_info.csv', names=cols)
area_passenger_info.shape

(997, 8)

In [5]:
### 将网格联系强度添加至表格中
### 在每个时刻，计算某一区域与其他所有区域的网格联系强度之和（in），然后减去该区域去往其他区域的网格联系强度之和（out）
cols = ['hour', 'start_grid_x', 'start_grid_y', 'end_grid_x', 'end_grid_y', 'index']
grid_strength = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/datafountain_competition_od.txt', names=cols, encoding='gb2312', sep='\t')
grid_strength.shape

(20, 6)

In [6]:
### 根据每个网格中心点与重点区域的距离，给每条网格联系强度打上id，以便后续处理
def add_id_to_grid(df, area_passenger_info):
    start_id = np.zeros((df.shape[0], 1))-999
    start_id = pd.DataFrame(start_id)
    start_id.columns = ['start_id']
    
    end_id = np.zeros((df.shape[0], 1))-999
    end_id = pd.DataFrame(end_id)
    end_id.columns = ['end_id']
    
    area_passenger_info['dis'] = 0
    
    for index, row in tqdm(df.iterrows()):
        if (start_id.iloc[index]['start_id']==-999):
            ### 遍历找出最近的出发地区
            ### 批量计算距离
            area_passenger_info['dis'] = (row['start_grid_x']-area_passenger_info['Grid_x'])**2 + (row['start_grid_y']-area_passenger_info['Grid_y'])**2
            ### 找出最小距离
            min_dis = area_passenger_info['dis'].min()
            ### 将最小距离对应的id赋给start_id
            start_id.iloc[index]['start_id'] = area_passenger_info[area_passenger_info['dis']==min_dis]['id'].values[0]
            ### 以出发地坐标为依据，给所有同地段的数据打上相同id
            start_id[(df['start_grid_x']==row['start_grid_x']) & (df['start_grid_y']==row['start_grid_y'])]['start_id'] = start_id.iloc[index]['start_id']
    
    for index, row in tqdm(df.iterrows()):
        if (end_id.iloc[index]['end_id']==-999):
            ### 遍历找出最近的出发地区
            ### 批量计算距离
            area_passenger_info['dis'] = (row['end_grid_x']-area_passenger_info['Grid_x'])**2 + (row['end_grid_y']-area_passenger_info['Grid_y'])**2
            ### 找出最小距离
            min_dis = area_passenger_info['dis'].min()
            ### 将最小距离对应的id赋给start_id
            end_id.iloc[index]['end_id'] = area_passenger_info[area_passenger_info['dis']==min_dis]['id'].values[0]
            ### 以出发地坐标为依据，给所有同地段的数据打上相同id
            end_id[(df['end_grid_x']==row['end_grid_x']) & (df['end_grid_y']==row['end_grid_y'])]['end_id'] = end_id.iloc[index]['end_id']
            
            
            
    ### 合并
    df = pd.concat([df, start_id], axis=1)
    df = pd.concat([df, end_id], axis=1)

    return df

In [7]:
grid_strength = add_id_to_grid(df=grid_strength, area_passenger_info=area_passenger_info)

20it [00:00, 21.12it/s]
20it [00:00, 21.63it/s]


In [8]:
### 保存到本地
grid_strength.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/grid_strength_processed.csv', index=None)